In [2]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
import math
import random
import numpy as np
from scipy.optimize import minimize
from qiskit import quantum_info

In [12]:
def get_u(param, b):
    qc = QuantumCircuit(2)
    qc.ry(param[0], 0)
    qc.ry(param[1], 1)
    qc.cz(0, 1)
    qc.ry(param[2], 0)
    qc.ry(param[3], 1)
    
    backend = Aer.get_backend('statevector_simulator')
    job = execute(qc, backend)
    result = job.result()
    outputstate = result.get_statevector(qc, decimals=8)
    
    circ = QuantumCircuit(2)
    circ.initialize(b, range(0, 2))
    backend = Aer.get_backend('statevector_simulator')
    job = execute(circ, backend)
    result = job.result()
    outputstate1 = result.get_statevector(circ, decimals=8)
    return abs(1 - quantum_info.state_fidelity(outputstate, outputstate1))

def generate_ansatz(noq, qc, angle):
    depth = noq
    off = 0
    for j in range(depth):
        for i in range(noq):
            qc.ry(angle[off + i], i)
            if (i != 0 and j != (depth - 1)):
                qc.cz(0,i)
        off += i+1
def get_angles_for_control_b(b, noq):
    b = b/np.linalg.norm(b)
    out = minimize(get_u, x0 = ([0]*(2**noq)), args = b, method="cobyla", options={'maxiter':200},tol = 0.0000000001)
    return out.x

def get_controlled_b(qc, b, noq, ancilla, reg, angles):
    generate_control_ansatz(noq, qc, angles, ancilla, reg)

def had_test_b(qc, ancilla_index, gate, noq, angle, b):
    qc.h(ancilla_index)
    generate_ansatz(noq, qc, angle)
    append_A_matrix(qc, gate, ancilla_index)
    generate_B(noq, qc, b)
    qc.h(ancilla_index)
    
def append_A_matrix(qc, gate, ancilla, start = 0, end = 0):
    if (end == 0):
        end = len(gate)
    for i in range(start, end):
        if (gate[i - start] == 1):
            qc.cx(ancilla, i)
        if (gate[i - start] == 2):
            qc.cy(ancilla, i)   
        if (gate[i - start] == 3):
            qc.cz(ancilla, i)             

def had_test_a(qc, ancilla_index, gat, gat_trans, noq, angle, bIm = False):
    qc.h(ancilla_index)
    if (bIm):
        qc.sdg(ancilla_index)
    generate_ansatz(noq, qc, angle)
    append_A_matrix(qc, gat, ancilla_index)
    append_A_matrix(qc, gat_trans, ancilla_index)
    qc.h(ancilla_index)

def generate_control_ansatz(noq, qc, angle, ancilla, reg):
    depth = noq
    off = 0
    for j in range(depth):
        for i in range(noq):
            qc.cry(angle[i+off], qiskit.circuit.Qubit(reg,ancilla), qiskit.circuit.Qubit(reg, i))
            if (i != 0 and j != (depth - 1)):
                qc.h(j)
                qc.ccx(ancilla, i, j)
                qc.h(j)
        off += i+1

def hadamard_test_num(noq, qc, angle, ancilla, reg, gate, b, b_angle):
        qc.h(ancilla)
        qc.barrier()
        generate_control_ansatz(noq, qc, angle, ancilla, reg)
        qc.barrier()
        append_A_matrix(qc, gate, ancilla)
        qc.barrier()
        get_controlled_b(qc, b, noq, ancilla, reg, b_angle)
        qc.barrier()
        qc.h(ancilla)

In [4]:
import numpy as np
I = np.asmatrix('1 0;0 1')
X = np.asmatrix('0 1;1 0')
Y = np.asmatrix('0 0-1j;0+1j 0')
Z = np.asmatrix('1 0;0 -1')
pauli_i = [I, 0]
pauli_x = [X, 1]
pauli_y = [Y, 2]
pauli_z = [Z, 3]

def get_basis_set(num_of_qubit = 1):
    basis_set = [pauli_i, pauli_x, pauli_y, pauli_z]
    int_set = [pauli_i, pauli_x, pauli_y, pauli_z]
    temp_set = []
    for i in range (num_of_qubit-1):
        for j in range (len(basis_set)):
            for k in range (len(int_set)):
                a = basis_set[j]
                b = int_set[k]
                #temp_set.append()
                temp_set.append([np.kron(a[0],b[0]),[a[1],b[1]]])
        int_set = temp_set
        #print(int_set)
        temp_set = []
    return int_set

def get_decomposition(A, num_of_q):
    basis = get_basis_set(num_of_q)
    dec_list = []
    for i in range(len(basis)):
        b = basis[i]
        coeff = np.trace(A*b[0])/(2**num_of_q)
        dec_list.append([coeff, b[1]])
    return dec_list
def get_arbitary_decomposition(A):
    size = A.shape
    size = size[0]
    power = math.log2(size)
    p = int(power)
    if (p != power):
        p += 1
    ret = np.zeros([2**p,2**p], dtype = float)
    ret[:size, :size] = A
    for i in range(size, 2**p):
        ret[i,i] = 1
    return get_decomposition(ret, p)

In [14]:
a = np.matrix('1 2 3 4; 2 5 1 3; 3 1 6 2; 4 3 2 7')
b = [0.7071067, 0.7071067, 0.7071067, 0.7071067]
#d = np.linalg.norm(b)
a = a/(10)#/(d)#*np.linalg.norm(a))
#b = b/d
print(np.linalg.norm(b), np.linalg.norm(a))
dec_list = get_arbitary_decomposition(a)
gate = []
coeff = []
for i in range(len(dec_list)):
    if (abs(dec_list[i][0]) != 0):
        coeff.append(dec_list[i][0])
        gate.append(dec_list[i][1])
print(coeff)
print(len(coeff))

b_angle = get_angles_for_control_b(b, 2)
def calculate_cost(angle):  
    shots = 2000
    T = shots
    denom = 0
    numer = 0
    cof = 0
    noq = 2
    backend = Aer.get_backend('qasm_simulator')
    #print(gate)
    #print(coeff)
    for i in range(len(coeff)):
        for k in range(len(coeff)):
            circ = QuantumCircuit(noq + 1,1)
            cof = coeff[i]*np.conj(coeff[k])
            had_test_a(circ, noq, gate[i], gate[k], noq, angle)
            circ.measure(noq, 0)
            job = execute(circ, backend=backend, shots=shots,max_credits=3)    
            result = job.result()
            counts = result.get_counts(circ)
            m_sum = 0
            if "0" in list (counts):
                m_sum=m_sum+counts["0"]/T
            if "1" in list (counts):
                m_sum=m_sum-counts["1"]/T
            denom += cof*m_sum
    print("#denom computed###########")
    for i in range(len(coeff)):
        qc1 = QuantumRegister(noq + 1)
        cr = ClassicalRegister(1)
        qc = QuantumCircuit(qc1, cr)
        hadamard_test_num(noq, qc, angle, noq, qc1, gate[i], b, b_angle)
        qc.measure(noq, 0)
        #print(qc)
        job = execute(qc, backend=backend, shots=shots,max_credits=3)    
        result = job.result()
        counts = result.get_counts(qc)
        m_sum_i = 0
        if "0" in list (counts):
            m_sum_i=m_sum_i+counts["0"]/T
        if "1" in list (counts):
            m_sum_i=m_sum_i-counts["1"]/T
        
        for j in range(len(coeff)):
            qc1 = QuantumRegister(noq + 1)
            cr = ClassicalRegister(1)
            qc = QuantumCircuit(qc1, cr)
            hadamard_test_num(noq, qc, angle, noq, qc1, gate[j], b, b_angle)
            qc.measure(noq, 0)
            #print(qc)
            job = execute(qc, backend=backend, shots=shots,max_credits=3)    
            result = job.result()
            counts = result.get_counts(qc)
            m_sum = 0
            #print(counts)
            if "0" in list (counts):
                m_sum=m_sum+counts["0"]/T
            if "1" in list (counts):
                m_sum=m_sum-counts["1"]/T
            numer += m_sum*m_sum_i*(coeff[i]*np.conj(coeff[j]))
    print("numer = "+str(numer)+" denom= "+str(denom)+"cost = "+str(1-(numer/denom))+" angle = "+ str(angle))
    
    return abs(1-(numer/denom))  
angle = [2, 4.11, 1, 1.57]
maxiter = 2000000
iters = 0 
threshold = 0.00000047
k=0
out = minimize(calculate_cost, x0=angle, method="cobyla", options={'maxiter':200},tol = threshold)
print(out)

1.4142134 1.40356688476182
[0.475, 0.2, -0.125, 0.3, 0.25, (-0.15000000000000002+0j), -0.175, -0.07500000000000001]
8
#denom computed###########
numer = (0.05808369250000001+0j) denom= (0.38129500000000005+0j)cost = (0.8476673113993103+0j) angle = [2.   4.11 1.   1.57]
#denom computed###########
numer = (0.009707421250000018+0j) denom= (0.25652125+0j)cost = (0.9621574382239287+0j) angle = [3.   4.11 1.   1.57]
#denom computed###########
numer = (0.19785190312499995+0j) denom= (0.91646375+0j)cost = (0.784113770866551+0j) angle = [2.   5.11 1.   1.57]
#denom computed###########
numer = (0.04888762374999997+0j) denom= (0.359126875+0j)cost = (0.863870884767396+0j) angle = [2.   5.11 2.   1.57]
#denom computed###########
numer = (0.17662818437499994+0j) denom= (0.9290093749999997+0j)cost = (0.8098747018833905+0j) angle = [2.   5.11 1.   2.57]
#denom computed###########
numer = (0.39375821937500005+0j) denom= (1.3675237499999997+0j)cost = (0.712064803719131+0j) angle = [1.26360139 5.51877533

#denom computed###########
numer = (0.09159839125+0j) denom= (0.18265687500000005+0j)cost = (0.498522071780764+0j) angle = [ 0.76660743  6.59655719 -1.15612318  0.25068027]
#denom computed###########
numer = (0.09846247375000003+0j) denom= (0.17942999999999992+0j)cost = (0.45124854400044545+0j) angle = [ 0.7663518   6.59635531 -1.15641065  0.25101084]
#denom computed###########
numer = (0.09841886250000002+0j) denom= (0.16937062499999997+0j)cost = (0.418914215496341+0j) angle = [ 0.76672686  6.59606935 -1.15677237  0.25071167]
#denom computed###########
numer = (0.09663606187500001+0j) denom= (0.17771937499999998+0j)cost = (0.4562435194530702+0j) angle = [ 0.76631067  6.59612469 -1.15638816  0.25069676]
#denom computed###########
numer = (0.10167104375000004+0j) denom= (0.17691937500000002+0j)cost = (0.425325554366219+0j) angle = [ 0.76657487  6.59607127 -1.15628465  0.25095586]
#denom computed###########
numer = (0.10238294999999999+0j) denom= (0.18278999999999995+0j)cost = (0.4398875